# **Data Segmentation Analysis**

## **Why?**

- Groups data into meaningful categories for targeted insights.
- For customer segmentation, product categorization, or regional analysis.

## **Group the Data Based on a Specfic Range.**

### (Helps understand the correlation between two measures.)

- \[measure\] by \[measure\]

Examples:

- Total Products by Sales Range
- Total Customers by Age

## **Segment Products into Cost Ranges and Count How Many Products Fall into Each Segment.**

In [1]:
WITH product_segments AS (
    SELECT
        product_key, 
        product_name,
        cost, 
        CASE
            WHEN cost < 100 THEN 'Below 100'
            WHEN cost BETWEEN 100 AND 500 THEN '100-500'
            WHEN cost BETWEEN 500 AND 1000 THEN '500-1000'
            ELSE 'Above 1000'
        END AS cost_range
    FROM gold.dim_products
)
SELECT
    cost_range,
    COUNT(product_key) AS total_products
FROM product_segments
GROUP BY cost_range
ORDER BY total_products DESC;

(4 rows affected)

Total execution time: 00:00:00.098

cost_range,total_products
Below 100,110
100-500,101
500-1000,45
Above 1000,39


### **Group customer into three segments based on thier spending behavior:**

- VIP: Customers with at least 12 months of history and spending more than 5,000.
- Regular: Customers with at least 12 months of history by spending 5,000 or less.
- New: Cusotmers with a lifespan less than 12 months.

### **And find the total number of customers by each group.**

In [2]:
WITH customer_spending AS (
    SELECT
        c.customer_key,
        SUM(f.sales_amount) AS total_spending,
        MIN(order_date) AS first_order, 
        MAX(order_date) AS last_order,
        DATEDIFF(month, MIN(order_date), MAX(order_date)) AS lifespan
    FROM gold.fact_sales f
    LEFT JOIN gold.dim_customers c 
        ON f.customer_key = c.customer_key
    GROUP BY c.customer_key
)
SELECT
    customer_segment,
    COUNT(customer_key) AS total_customers
FROM (
    SELECT 
        customer_key, 
        CASE
            WHEN lifespan >= 12 AND total_spending > 5000 THEN 'VIP'
            WHEN lifespan >= 12 AND total_spending <= 5000 THEN 'Regular'
            ELSE 'New'
        END AS customer_segment
    FROM customer_spending
) AS segmented_customers
GROUP BY customer_segment
ORDER BY total_customers DESC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.709

customer_segment,total_customers
New,14631
Regular,2198
VIP,1655
